In [1]:
# !pip install "chonkie[semantic]"
# !pip install markitdown
# !pip install qdrant-client
# !pip install fastembed

In [2]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool
from crewai_tools import PDFSearchTool
# from tools.custom_tool import DocumentSearchTool
from tools.custom_tool import DocumentSearchTool


/Users/shkhilji/Documents/AI/ai-engineering-hub-main/agentic_rag_deepseek/agent_rag_venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
pdf_tool = DocumentSearchTool(file_path='/Users/shkhilji/Documents/AI/Docs/Actual scanned doc/Ibhar - Scanned 2.pdf')

In [5]:
web_search_tool = SerperDevTool()

In [6]:
import yaml
with open('config/agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

In [7]:
retriever_agent = Agent(
    config=agents_config['retriever_agent'],
    verbose=True,
    tools=[
        pdf_tool,
        web_search_tool
    ]
)

response_synthesizer_agent = Agent(
    config=agents_config['response_synthesizer_agent'],
    verbose=True
)

retrieval_task = Task(
    description=tasks_config['retrieval_task']['description'],
    expected_output=tasks_config['retrieval_task']['expected_output'],  
    agent=retriever_agent
)

response_task = Task(
    description=tasks_config['response_task']['description'],
    expected_output=tasks_config['response_task']['expected_output'], 
    agent=response_synthesizer_agent
)

In [ ]:
crew = Crew(
			agents=[retriever_agent, response_synthesizer_agent], 
			tasks=[retrieval_task, response_task],
			process=Process.sequential,
			verbose=True,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)

In [ ]:
result = crew.kickoff(inputs={"query": "Who is elon musk?"})
print(result)